In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms

import torch
import torch.nn as nn
import mat73
from pathlib import Path
import os
from torchsummary import summary

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float32)

In [22]:
path_dataset_folder = Path('./data/[1] simple linear')

path_file_input = path_dataset_folder / 'xTrain.mat'
path_file_output = path_dataset_folder / 'yTrain.mat'
path_train_data = (path_file_input, path_file_output)
torch.set_default_dtype(torch.float32)

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, path_train_data):
        self.input_data = mat73.loadmat(path_train_data[0])
        self.output_data = mat73.loadmat(path_train_data[1])
        # input_x = input_data['input'][:,0]
        # input_t = input_data['input'][:,1]
        # output_u = output_data['output']

    def __len__(self):
        return len(self.input_data['input'])

    def __getitem__(self, i):
        # input_data_sample = torch.from_numpy(self.input_data['input'][i, :], dtype=torch.float32)
        input_data_x = torch.from_numpy(np.asarray(self.input_data['input'][i, 0].astype('float32'))).unsqueeze(0)
        input_data_t = torch.from_numpy(np.asarray(self.input_data['input'][i, 1].astype('float32'))).unsqueeze(0)
        # output_data_sample = torch.from_numpy(self.output_data['output'][i], dtype=torch.float32)
        output_data_sample = torch.from_numpy(np.asarray(self.output_data['output'][i].astype('float32')))
        return input_data_x, input_data_t, output_data_sample

dataset_simple = MyDataset(path_train_data)
my_batch_size = int(np.power(2, 12))
train_dataloader=torch.utils.data.DataLoader(dataset_simple, batch_size=my_batch_size, shuffle=True)

In [29]:
for i, (train_input_x, train_input_t, train_output) in tqdm(enumerate(train_dataloader)):
    # print(train_output)
    print(train_output.size())

2it [00:00, 13.97it/s]

torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])


8it [00:00, 17.88it/s]

torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([3621])


In [33]:
train_input_x = train_input_x.to(device)
train_input_t = train_input_t.to(device)
train_output = train_output.to(device)

train_input_x.requires_grad = True
train_input_t.requires_grad = True


In [45]:
train_input_x.size()

torch.Size([3621, 1])

In [26]:
bb = torch.cat((train_input_x, train_input_t), dim=1)
# bb = torch.hstack((train_input_x, train_input_t))

In [27]:
bb.size()

torch.Size([3621, 2])

In [21]:
a = torch.tensor([1, 2, 3])

In [28]:
class my_bench_model(nn.Module):
    def __init__(self):
        super(my_bench_model, self).__init__()
        #You code after this line

        self.my_layer = nn.Sequential(nn.Linear(2, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 1))

    def forward(self, x, t):
        #You code after this line
        x = torch.cat((train_input_x, train_input_t), dim=1)
        x = self.my_layer(x)
        return x

model = my_bench_model().to(device)

In [34]:
model_predict = model(train_input_x, train_input_t)

In [35]:
model_predict.size()

torch.Size([3621, 1])

In [44]:
train_input_x.size()

torch.Size([3621, 1])

In [38]:
derivs = dict()

derivs['du_dx'], derivs['du_dt'] = torch.autograd.grad(model_predict, (train_input_x, train_input_t), create_graph=True, grad_outputs=torch.ones_like(model_predict))

In [39]:
derivs

{'du_dx': tensor([[0.0102],
         [0.0102],
         [0.0091],
         ...,
         [0.0098],
         [0.0103],
         [0.0107]], device='cuda:0', grad_fn=<SliceBackward0>),
 'du_dt': tensor([[0.0078],
         [0.0078],
         [0.0078],
         ...,
         [0.0078],
         [0.0078],
         [0.0077]], device='cuda:0', grad_fn=<SliceBackward0>)}

In [ ]:
def train_multiple_epochs(model, num_epochs, train_dataloader, test_dataloader, loss_function, optimizer,device,test_frequency=5):

    for epoch in range(1,num_epochs+1):
        print("Starting epoch number " + str(epoch))
        train_loss,model = train_classifier_one_epoch(train_dataloader, model, loss_function, optimizer,device)
        print("Loss for Training on Epoch " +str(epoch) + " is "+ str(train_loss))
        if(epoch%test_frequency==0 or epoch==1 or epoch==num_epochs):
            test_classifier(test_dataloader, model)
    return model